In [1]:
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

#聲音處理套件
import librosa
#資料處理套件
import pandas as pd
import numpy as np
#時間套件
import time


In [2]:
#存檔位置
ModelMode = "LeNet5_ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
filePath_Train = ModelMode + TrainDataSoundMode[3] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[3] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[3]

In [3]:
#音訊資訊CSV檔位置
DATA_CSV_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/audio/"

def loadDataESC50(data_path, data_csv_path, soundLength):
    dataX_Fold = []
    dataY_Fold = []
    
    #讀取CSV檔
    df = pd.read_csv(data_csv_path)
    #篩選出ESC10作為訓練資料
    df_10 = df[df['esc10']==True]
    #取出類別，去除重複並排序
    classes = df_10['category'].unique()
    class_dict = {i:x for x,i in enumerate(classes)}
    #重新標label
    df_10['target'] = df_10['category'].map(class_dict)

    #計數
    count = 0
    
    for Fold in range(1, 6, 1):
        #初始化
        dataX = []
        dataY = []
        #篩選資料
        sound_df = df_10[df_10.fold == Fold]
        #轉換train資料/添加標籤
        for data in sound_df.itertuples():
            sig, sr = librosa.load(data_path+data[1])
            for i in range((6-soundLength)):
                sound = sig[i*sr : int((i+soundLength)*sr)]
                sound = audio_to_melspectrogram(audio=sound, sr=sr)
                dataX.append(sound)
                dataY.append(data[3])       
            count +=1
            print("{:.3f}".format(100 * count / len(df_10))+"%", end='\r')
        dataX_Fold.append(dataX)
        dataY_Fold.append(dataY)
                
    #通通轉成 torch 的 tensor 形式
    dataX_Fold = torch.Tensor(dataX_Fold)
    dataY_Fold = torch.Tensor(dataY_Fold).type(torch.LongTensor)
    
    #轉維度 FBCHW
    dataX_Fold = torch.reshape(dataX_Fold, (dataX_Fold.shape[0], dataX_Fold.shape[1], 1, dataX_Fold.shape[2], dataX_Fold.shape[3]))
           
    # 完成時打印最終消息
    print("Progress: 100% - Completed")
    return dataX_Fold, dataY_Fold

class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

#轉頻譜圖
def audio_to_melspectrogram(audio, sr):
    spectrogram = librosa.feature.melspectrogram(y=audio, 
                                                 sr=sr,
                                                 n_mels=128,
                                                 hop_length=512,
                                                 n_fft=2048,
                                                 fmin=20,
                                                 fmax=sr//2)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

class CustomDatasetESC50(Dataset):
    def __init__(self, inputDataX, inputDataY, fold, isTrain):
        #防止範圍錯誤
        if(fold>5 or fold <1):
            raise ValueError("Fold範圍錯誤")
            
        #資料初始化
        self.dataX = []
        self.dataY = []
        
        # True:訓練資料  False:測試資料
        if isTrain:
            for Fold in range(1, 6, 1):
                if(Fold != fold):
                    for X in inputDataX[Fold-1]:
                        self.dataX.append(X)
                    for Y in inputDataY[Fold-1]:
                        self.dataY.append(Y)
        else:
            self.dataX = inputDataX[fold-1]
            self.dataY = inputDataY[fold-1]

    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, index):
        # 根據索引返回單個樣本及其標籤
        data = self.dataX[index]
        label = self.dataY[index]
        
        return data, label

In [4]:
#LeNet5模型
class LeNet(nn.Module):
    def __init__(self, nums_class=10):
        super(LeNet,self).__init__()
        
        #卷基層
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        
        #全連接層
        self.fullconn_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(32,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,nums_class)           
        )
        
    def forward(self,x):
        output = self.conv_layer(x)
        output = self.fullconn_layer(output)
        return output

In [5]:
#訓練模型(全部訓練)
def train_model_TrainAll(trainFold):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    
    #---------------------------------------------------
    #訓練相關參數
    #---------------------------------------------------
    #訓練次數
    num_epochs = 60
    #批次大小
    BATCH_SIZE = 64
    #學習率
    LR = 0.0032
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #scheduler
    #--------------------------------------------------------------------
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #資料抓入
    #預處理
    #--------------------------------------------------------------------
    #資料抓入
    trainloader = CustomDatasetESC50(inputDataX = dataX,
                               inputDataY = dataY,
                               fold = trainFold,
                               isTrain = True)
    #分批方法
    train_loader = torch.utils.data.DataLoader(trainloader, batch_size = BATCH_SIZE, shuffle = True)
    #--------------------------------------------------------------------
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(inputs)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        
    #測試準確度
    test_accuracy1 = test_model(inputDataX = dataX1, inputDataY = dataY1, testFold = trainFold)
    test_accuracy3 = test_model(inputDataX = dataX3, inputDataY = dataY3, testFold = trainFold)
    test_accuracy5 = test_model(inputDataX = dataX5, inputDataY = dataY5, testFold = trainFold)
        
    #將gpu資料取回
    train_loss_array = torch.stack(train_loss_array,dim=-1).cpu()
    train_accuracy_array = torch.stack(train_accuracy_array,dim=-1).cpu()
    
    return train_loss_array, train_accuracy_array, test_accuracy1, test_accuracy3, test_accuracy5

In [6]:
#測試模型的準確度
def test_model(inputDataX, inputDataY, testFold):
    #固定模型不改變權重
    model.eval()
    #計數初始化
    correct = 0
    total = 0
    #資料抓取
    testloader = CustomDatasetESC50(inputDataX = inputDataX,
                               inputDataY = inputDataY,
                               fold = testFold,
                               isTrain = False)
    #封裝                                
    test_loader = torch.utils.data.DataLoader(testloader, batch_size=64, shuffle=False)
    
    #不計算梯度 加速計算
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #預測結果
            outputs = model(inputs)
            
            #獲取預測標籤
            _, predicted = torch.max(outputs.data, 1)
            
            #計算準確度
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
    return (100 * correct / total)

In [7]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
dataX1, dataY1 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 1)
print("讀取3秒資料")
dataX3, dataY3 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 3)
print("讀取5秒資料")
dataX5, dataY5 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_1644\4284335736.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)


C:\Users\test\AppData\Local\Temp\ipykernel_1644\4284335736.py:43: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)
C:\Users\test\AppData\Local\Temp\ipykernel_1644\4284335736.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_1644\4284335736.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)


Progress: 100% - Completed
Data Loading Time: 28.04 seconds


In [8]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = LeNet(nums_class = 10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        #訓練資料(要用幾秒訓練)
        if(TL==0):
            #5s
            dataX = dataX5
            dataY = dataY5
        elif(TL==1):
            #3s
            dataX = dataX3
            dataY = dataY3
        else:
            #1s
            dataX = dataX1
            dataY = dataY1
    

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型(只訓練分類器)
        train_loss, train_accuracy, test_accuracy1, test_accuracy3, test_accuracy5 = train_model_TrainAll(fold)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/60 Train_Loss: 3.0723724365234375 Train_acc: 13.750000%
Train Epoch: 2/60 Train_Loss: 2.080845594406128 Train_acc: 18.750000%
Train Epoch: 3/60 Train_Loss: 1.9423869848251343 Train_acc: 20.625000%
Train Epoch: 4/60 Train_Loss: 1.8676753044128418 Train_acc: 25.312500%
Train Epoch: 5/60 Train_Loss: 1.7986587285995483 Train_acc: 34.062500%
Train Epoch: 6/60 Train_Loss: 1.7260868549346924 Train_acc: 32.500000%
Train Epoch: 7/60 Train_Loss: 1.6384679079055786 Train_acc: 32.500000%
Train Epoch: 8/60 Train_Loss: 1.8304948806762695 Train_acc: 24.375000%
Train Epoch: 9/60 Train_Loss: 1.791573166847229 Train_acc: 29.375000%
Train Epoch: 10/60 Train_Loss: 1.776981234550476 Train_acc: 32.187500%
Train Epoch: 11/60 Train_Loss: 1.705

Train Epoch: 48/60 Train_Loss: 0.37153902649879456 Train_acc: 88.125008%
Train Epoch: 49/60 Train_Loss: 0.2678147256374359 Train_acc: 87.812508%
Train Epoch: 50/60 Train_Loss: 0.33444923162460327 Train_acc: 88.020836%
Train Epoch: 51/60 Train_Loss: 0.3144564926624298 Train_acc: 87.291672%
Train Epoch: 52/60 Train_Loss: 0.3194352686405182 Train_acc: 88.125008%
Train Epoch: 53/60 Train_Loss: 0.4117804169654846 Train_acc: 87.395836%
Train Epoch: 54/60 Train_Loss: 0.4258394241333008 Train_acc: 87.916672%
Train Epoch: 55/60 Train_Loss: 0.3485260605812073 Train_acc: 87.395836%
Train Epoch: 56/60 Train_Loss: 0.3664611577987671 Train_acc: 87.708336%
Train Epoch: 57/60 Train_Loss: 0.31101447343826294 Train_acc: 88.125008%
Train Epoch: 58/60 Train_Loss: 0.3106728792190552 Train_acc: 87.916672%
Train Epoch: 59/60 Train_Loss: 0.35939574241638184 Train_acc: 87.604172%
Train Epoch: 60/60 Train_Loss: 0.3522692620754242 Train_acc: 88.854172%
Test accuracy of the model: 55.250000%
Test accuracy of the 

Train Epoch: 32/60 Train_Loss: 1.1437091827392578 Train_acc: 59.375000%
Train Epoch: 33/60 Train_Loss: 1.073057770729065 Train_acc: 61.875000%
Train Epoch: 34/60 Train_Loss: 1.0942879915237427 Train_acc: 63.750000%
Train Epoch: 35/60 Train_Loss: 0.8338758945465088 Train_acc: 64.375000%
Train Epoch: 36/60 Train_Loss: 0.9429715275764465 Train_acc: 66.562500%
Train Epoch: 37/60 Train_Loss: 0.8316797018051147 Train_acc: 65.312500%
Train Epoch: 38/60 Train_Loss: 0.991879403591156 Train_acc: 64.375000%
Train Epoch: 39/60 Train_Loss: 1.0925931930541992 Train_acc: 65.000000%
Train Epoch: 40/60 Train_Loss: 0.9748331308364868 Train_acc: 65.937500%
Train Epoch: 41/60 Train_Loss: 0.9090149998664856 Train_acc: 65.625000%
Train Epoch: 42/60 Train_Loss: 0.8238962888717651 Train_acc: 67.812500%
Train Epoch: 43/60 Train_Loss: 0.7431831955909729 Train_acc: 67.187500%
Train Epoch: 44/60 Train_Loss: 0.8889331221580505 Train_acc: 68.125000%
Train Epoch: 45/60 Train_Loss: 0.9141428470611572 Train_acc: 66.87

Train Epoch: 17/60 Train_Loss: 0.45014166831970215 Train_acc: 76.312500%
Train Epoch: 18/60 Train_Loss: 0.6649384498596191 Train_acc: 77.375000%
Train Epoch: 19/60 Train_Loss: 0.4559873640537262 Train_acc: 77.500000%
Train Epoch: 20/60 Train_Loss: 0.43240660429000854 Train_acc: 78.125000%
Train Epoch: 21/60 Train_Loss: 0.5321067571640015 Train_acc: 76.687500%
Train Epoch: 22/60 Train_Loss: 0.7001663446426392 Train_acc: 78.562500%
Train Epoch: 23/60 Train_Loss: 0.6365081667900085 Train_acc: 79.437500%
Train Epoch: 24/60 Train_Loss: 0.5052174925804138 Train_acc: 77.312500%
Train Epoch: 25/60 Train_Loss: 0.6306679248809814 Train_acc: 74.375000%
Train Epoch: 26/60 Train_Loss: 0.5400495529174805 Train_acc: 77.562500%
Train Epoch: 27/60 Train_Loss: 0.5572203397750854 Train_acc: 79.937500%
Train Epoch: 28/60 Train_Loss: 0.533606231212616 Train_acc: 80.812500%
Train Epoch: 29/60 Train_Loss: 0.43627816438674927 Train_acc: 76.937500%
Train Epoch: 30/60 Train_Loss: 0.6672844886779785 Train_acc: 7

Train Epoch: 1/60 Train_Loss: 1.5467026233673096 Train_acc: 38.958336%
Train Epoch: 2/60 Train_Loss: 1.342357873916626 Train_acc: 45.208336%
Train Epoch: 3/60 Train_Loss: 1.1574301719665527 Train_acc: 49.375004%
Train Epoch: 4/60 Train_Loss: 1.0407023429870605 Train_acc: 53.020836%
Train Epoch: 5/60 Train_Loss: 0.826680600643158 Train_acc: 57.395836%
Train Epoch: 6/60 Train_Loss: 1.0358233451843262 Train_acc: 60.520836%
Train Epoch: 7/60 Train_Loss: 0.9811044931411743 Train_acc: 59.687504%
Train Epoch: 8/60 Train_Loss: 1.165197730064392 Train_acc: 53.437504%
Train Epoch: 9/60 Train_Loss: 0.891255795955658 Train_acc: 59.583336%
Train Epoch: 10/60 Train_Loss: 0.6275896430015564 Train_acc: 66.666672%
Train Epoch: 11/60 Train_Loss: 0.8298185467720032 Train_acc: 66.666672%
Train Epoch: 12/60 Train_Loss: 0.8767989277839661 Train_acc: 67.500000%
Train Epoch: 13/60 Train_Loss: 0.8087087869644165 Train_acc: 69.062500%
Train Epoch: 14/60 Train_Loss: 0.7613731622695923 Train_acc: 69.479172%
Train

Train Epoch: 51/60 Train_Loss: 0.28464359045028687 Train_acc: 89.312500%
Train Epoch: 52/60 Train_Loss: 0.24748371541500092 Train_acc: 89.250000%
Train Epoch: 53/60 Train_Loss: 0.2510641813278198 Train_acc: 89.437500%
Train Epoch: 54/60 Train_Loss: 0.3717744052410126 Train_acc: 89.062500%
Train Epoch: 55/60 Train_Loss: 0.22394472360610962 Train_acc: 89.750000%
Train Epoch: 56/60 Train_Loss: 0.30432188510894775 Train_acc: 90.375000%
Train Epoch: 57/60 Train_Loss: 0.2505630850791931 Train_acc: 89.312500%
Train Epoch: 58/60 Train_Loss: 0.1718532145023346 Train_acc: 88.812500%
Train Epoch: 59/60 Train_Loss: 0.5138238072395325 Train_acc: 89.875000%
Train Epoch: 60/60 Train_Loss: 0.24510757625102997 Train_acc: 89.875000%
Test accuracy of the model: 68.500000%
Test accuracy of the model: 73.333333%
Test accuracy of the model: 76.250000%
Training Time: 9.78 seconds
可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fol

Train Epoch: 35/60 Train_Loss: 0.3031647801399231 Train_acc: 83.125008%
Train Epoch: 36/60 Train_Loss: 0.38399738073349 Train_acc: 83.958336%
Train Epoch: 37/60 Train_Loss: 0.5640456676483154 Train_acc: 83.333336%
Train Epoch: 38/60 Train_Loss: 0.491637647151947 Train_acc: 83.437508%
Train Epoch: 39/60 Train_Loss: 0.50312340259552 Train_acc: 83.645836%
Train Epoch: 40/60 Train_Loss: 0.4675767123699188 Train_acc: 82.291672%
Train Epoch: 41/60 Train_Loss: 0.4168214499950409 Train_acc: 83.854172%
Train Epoch: 42/60 Train_Loss: 0.45951175689697266 Train_acc: 81.458336%
Train Epoch: 43/60 Train_Loss: 0.44584277272224426 Train_acc: 83.541672%
Train Epoch: 44/60 Train_Loss: 0.38619697093963623 Train_acc: 83.854172%
Train Epoch: 45/60 Train_Loss: 0.5430933833122253 Train_acc: 82.916672%
Train Epoch: 46/60 Train_Loss: 0.4193130433559418 Train_acc: 84.375008%
Train Epoch: 47/60 Train_Loss: 0.4056243300437927 Train_acc: 86.041672%
Train Epoch: 48/60 Train_Loss: 0.34243670105934143 Train_acc: 86.3

Train Epoch: 18/60 Train_Loss: 1.123448133468628 Train_acc: 47.187500%
Train Epoch: 19/60 Train_Loss: 1.425771951675415 Train_acc: 47.812500%
Train Epoch: 20/60 Train_Loss: 1.4036275148391724 Train_acc: 48.437500%
Train Epoch: 21/60 Train_Loss: 1.1455910205841064 Train_acc: 49.375000%
Train Epoch: 22/60 Train_Loss: 1.2085130214691162 Train_acc: 51.875000%
Train Epoch: 23/60 Train_Loss: 1.1506718397140503 Train_acc: 53.125000%
Train Epoch: 24/60 Train_Loss: 1.0764825344085693 Train_acc: 56.250000%
Train Epoch: 25/60 Train_Loss: 1.1483988761901855 Train_acc: 59.062500%
Train Epoch: 26/60 Train_Loss: 0.9739882349967957 Train_acc: 62.187500%
Train Epoch: 27/60 Train_Loss: 1.0939972400665283 Train_acc: 59.375000%
Train Epoch: 28/60 Train_Loss: 0.9097897410392761 Train_acc: 62.187500%
Train Epoch: 29/60 Train_Loss: 1.0933260917663574 Train_acc: 56.875000%
Train Epoch: 30/60 Train_Loss: 1.0750954151153564 Train_acc: 59.062500%
Train Epoch: 31/60 Train_Loss: 1.021034836769104 Train_acc: 64.062

Train Epoch: 3/60 Train_Loss: 0.7996869087219238 Train_acc: 58.750000%
Train Epoch: 4/60 Train_Loss: 0.970420777797699 Train_acc: 64.375000%
Train Epoch: 5/60 Train_Loss: 1.012830138206482 Train_acc: 66.312500%
Train Epoch: 6/60 Train_Loss: 0.7783952355384827 Train_acc: 64.812500%
Train Epoch: 7/60 Train_Loss: 0.983619213104248 Train_acc: 64.875000%
Train Epoch: 8/60 Train_Loss: 0.8753712177276611 Train_acc: 62.437500%
Train Epoch: 9/60 Train_Loss: 0.8177152872085571 Train_acc: 66.437500%
Train Epoch: 10/60 Train_Loss: 0.8188760876655579 Train_acc: 67.500000%
Train Epoch: 11/60 Train_Loss: 0.9129131436347961 Train_acc: 70.937500%
Train Epoch: 12/60 Train_Loss: 0.6009199619293213 Train_acc: 70.687500%
Train Epoch: 13/60 Train_Loss: 0.7946892976760864 Train_acc: 72.812500%
Train Epoch: 14/60 Train_Loss: 0.761273205280304 Train_acc: 69.062500%
Train Epoch: 15/60 Train_Loss: 0.6650470495223999 Train_acc: 70.062500%
Train Epoch: 16/60 Train_Loss: 0.7187983393669128 Train_acc: 75.062500%
Tra

In [9]:
def getMax(data):
    TL2 = []
    for fold in range(len(data)):
        TL2.append(data[fold][-1])
        
    return max(TL2)

def getMin(data):
    TL2 = []
    for fold in range(len(data)):
        TL2.append(data[fold][-1])
        
    return min(TL2)

def getMean(data):
    TL2 = []
    for fold in range(len(data)):
        TL2.append(data[fold][-1])
        
    return np.mean(TL2)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(getMax(kFoldTrainAccuracy), getMax(kFoldTestAccuracy1), getMax(kFoldTestAccuracy3), getMax(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(getMin(kFoldTrainAccuracy), getMin(kFoldTestAccuracy1), getMin(kFoldTestAccuracy3), getMin(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(getMean(kFoldTrainAccuracy), getMean(kFoldTestAccuracy1), getMean(kFoldTestAccuracy3), getMean(kFoldTestAccuracy5)))

f.close()